In [51]:
from pathlib import Path
from ultralytics import YOLO
import numpy as np

In [52]:
def iou_calculator(box1, box2):                                                     

    x1, y1, width_1, height_1 = box1 [1], box1 [2], box1 [3], box1 [4]                
    x2, y2, width_2, height_2 = float(box2 [1]), float(box2 [2]), float(box2 [3]), float(box2 [4])

    x_left = max(x1 - width_1 / 2, x2 - width_2 / 2)                                # Calculating coordinates of intersection rectangle
    y_top = max(y1 - height_1 / 2, y2 - height_2 / 2)
    x_right = min(x1 + width_1 / 2, x2 + width_2 / 2)
    y_bottom = min(y1 + height_1 / 2, y2 + height_2 / 2)
    
    intersection_area = max(0, x_right - x_left) * max(0, y_bottom - y_top)         # Calculating area of intersection rectangle
    
    box1_area = width_1 * height_1                                                    # Calculating area of both bounding boxes
    box2_area = width_2 * height_2
    
    union_area = box1_area + box2_area - intersection_area                          # Calculating union area
    
    if union_area == 0:                                                              
        iou = 0  # Avoid division by zero
    else:
        iou = intersection_area / union_area
    
    return iou



In [53]:
def confusion_matrix_calculator_for_oneframe(list_of_gt_boxes, list_of_pred_boxes, iou_threshold):         #Confusion_matrix_calculator for singel frame
    box_local_TP = 0  
    box_local_FP = 0 
    box_local_FN = 0  

    class_local_TP = 0  
    class_local_FP = 0 
    class_local_FN = 0  

    for actual_box in list_of_gt_boxes:
        for pred_box in list_of_pred_boxes:
            gt_matched_box = False
            gt_matched_class = False                                                 # Flags to check the match
            
            iou = iou_calculator(actual_box, pred_box)                               # Calculatig IoU between ground truth box and predicted box
            if iou >= iou_threshold:
                box_local_TP += 1
                gt_matched_box = True
                if actual_box[0] == actual_box[0]:
                     class_local_TP += 1
                     gt_matched_class = True  
                break                 
        if not gt_matched_box:                                                   # If ground truth box is not matched with any predicted box, consider it a false negative
            box_local_FN += 1
        if not gt_matched_class:
            class_local_FN += 1

    box_local_FP = len(list_of_pred_boxes) - box_local_TP
    class_local_FP = len(list_of_pred_boxes) - class_local_TP


    return box_local_TP, box_local_FP, box_local_FN, class_local_TP, class_local_FP, class_local_FN
    

In [54]:
def class_box_generator_for_gt(txt_file):
    list_of_detections_gt = []
    with open(txt_file, 'r') as file:
        for line in file:
            detection = line.strip().split()
            list_of_detections_gt.append(detection)
    return list_of_detections_gt

In [55]:
def class_box_generator_for_pred(prediction_results):

    for result in prediction_results:
        list_of_pred_boxes = []
        pred_class_list = result.boxes.cls.tolist()
        box_array = result.boxes.xywhn.tolist()
        
        for detection in box_array:
            detection_data = np.append(pred_class_list[box_array.index(detection)], detection)
            list_of_pred_boxes.append(detection_data)
      
        return list_of_pred_boxes

In [56]:
def precision_recall_calculator(path_to_test_images, path_to_gt_txts):

    model = YOLO('yolov8s.pt')

    list_of_images = list(image for image in (path_to_test_images.iterdir()))
    list_of_gt_txts = list(json for json in (path_to_gt_txts.iterdir()))

    list_of_images.sort()
    list_of_gt_txts.sort()

    combined_box_TP = 0                 #for all frames                                                                    
    combined_box_FP = 0 
    combined_box_FN = 0  


    combined_class_TP = 0             #for all frames                                                               
    combined_class_FP = 0 
    combined_class_FN = 0 


    for image in list_of_images:
        gt_txt = list_of_gt_txts[list_of_images.index(image)]
        list_of_gt_boxes = class_box_generator_for_gt(gt_txt)
        
        results = model.predict(image, save=True)
        list_of_pred_boxes = class_box_generator_for_pred(results)
        

        #BBOX C-Matrix Recording 
        box_local_TP, box_local_FP, box_local_FN, class_local_TP, class_local_FP, class_local_FN = confusion_matrix_calculator_for_oneframe(list_of_pred_boxes, list_of_gt_boxes, iou_threshold=0.8)

        combined_box_TP += box_local_TP
        combined_box_FP += box_local_FP
        combined_box_FN += box_local_FN

        combined_class_TP += class_local_TP
        combined_class_FP += class_local_FP
        combined_class_FN += class_local_FN


    bbox_precision = combined_box_TP / (combined_box_TP + combined_box_FP) if (combined_box_TP + combined_box_FP) > 0 else 0                     
    bbox_recall = combined_box_TP / (combined_box_TP + combined_box_FN) if (combined_box_TP + combined_box_FN) > 0 else 0

    class_precision = combined_class_TP / (combined_class_TP + combined_class_FP) if (combined_class_TP + combined_class_FP) > 0 else 0                    
    class_recall = combined_class_TP / (combined_class_TP + combined_class_FN) if (combined_class_TP + combined_class_FN) > 0 else 0
    
    print(f'bbox precision: {bbox_precision * 100}%, bbox_recall {bbox_recall}')
    print(f'class precision: {class_precision * 100}%, class_recall {class_recall}')





In [57]:
path_to_test_images = Path(r'C:\Users\hussa\OneDrive\Desktop\Project\Dataset\INFRA-3DRC_scene-4\INFRA-3DRC_scene-4\camera_01\camera_01__data')
path_to_gt_txts = Path(r'C:\Users\hussa\OneDrive\Desktop\Project\Dataset\INFRA-3DRC_scene-4\INFRA-3DRC_scene-4\camera_01\labels')

precision_recall_calculator(path_to_test_images, path_to_gt_txts)


image 1/1 C:\Users\hussa\OneDrive\Desktop\Project\Dataset\INFRA-3DRC_scene-4\INFRA-3DRC_scene-4\camera_01\camera_01__data\camera_01__2023-07-06-20-05-14-905.png: 416x640 1 person, 2 cars, 466.5ms
Speed: 0.0ms preprocess, 466.5ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs\detect\predict

image 1/1 C:\Users\hussa\OneDrive\Desktop\Project\Dataset\INFRA-3DRC_scene-4\INFRA-3DRC_scene-4\camera_01\camera_01__data\camera_01__2023-07-06-20-05-14-972.png: 416x640 1 person, 2 cars, 434.8ms
Speed: 5.0ms preprocess, 434.8ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs\detect\predict

image 1/1 C:\Users\hussa\OneDrive\Desktop\Project\Dataset\INFRA-3DRC_scene-4\INFRA-3DRC_scene-4\camera_01\camera_01__data\camera_01__2023-07-06-20-05-15-005.png: 416x640 1 person, 1 car, 431.6ms
Speed: 17.1ms preprocess, 431.6ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs\detect\predict

ima